In [1]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.manifold import TSNE
import plotly.express as px
from boolrank import DualSiglip2Model
from my_processing import paths_to_dataset

# Load data
data_path = "training"
paths = [
    f"data/{data_path}.jsonl",
    "data/TAR_data.jsonl",
    "data/sysrev_conv.jsonl",
]
dataset = paths_to_dataset(
    paths,
    test_only_sources=['TAR', 'sysrev'],
    # train_sources=['pubmed-searchrefiner','pubmed-query','raw-jsonl']
    train_sources=['pubmed-searchrefiner','raw-jsonl']
)

data/training.jsonl
data/TAR_data.jsonl
data/sysrev_conv.jsonl


Finding similar: 100%|██████████| 3782/3782 [00:03<00:00, 1073.54it/s]


In [2]:
N = 10000
df = pd.DataFrame({
    "nl": dataset["train"]["nl_query"],
    "bool": dataset["train"]["bool_query"],
    "quality": dataset["train"]["quality"],
    "source": dataset["train"]["source"]
})
df = df.sample(min(N, df.shape[0])).reset_index(drop=True)
df

,nl,bool,quality,source
0,Why not just Google it? An assessment of infor...,"((((""information""[All Fields]) AND (""seeking""[...",0.250530,pubmed-searchrefiner
1,Implementation of Developmental Screening Guid...,((Neurodevelopmental Disorders[MeSH Terms] OR ...,0.043195,pubmed-searchrefiner
2,Early erythropoiesis-stimulating agents in pre...,"((infant, newborn[MeSH] OR newborn OR neonate...",0.013400,raw-jsonl
3,Improvements in gait speed and weight shift of...,"(((""benign paroxysmal positional vertigo""[tia...",0.313163,pubmed-searchrefiner
4,The effect of pharmacist-provided non-dispensi...,((Pharmacy [mh] OR Pharmacy [tiab] OR pharmac...,0.549417,raw-jsonl
...,...,...,...,...
3777,Sildenafil for pulmonary hypertension in neonates,"((infant, newborn[MeSH] OR newborn OR neonate...",0.013400,raw-jsonl
3778,Randomized trial of three phototherapy methods...,"((""Acne Vulgaris""[Mesh] OR Acne[tiab]) AND (""P...",0.062633,pubmed-searchrefiner
3779,Transfusion of red blood cells stored for shor...,(red cell*[TI] OR blood[TI] OR erythrocyte*[T...,0.549417,raw-jsonl
3780,The clinical and histological effect of home-u...,"((""Acne Vulgaris""[Mesh] OR Acne[tiab] OR Black...",0.003641,pubmed-searchrefiner


In [ ]:
import torch

model = DualSiglip2Model('BAAI/bge-small-en-v1.5', loss_type="clip")

embeddings = model.encode_bool(df["bool"].tolist(), batch_size=100).detach().cpu().numpy()
torch.cuda.empty_cache()

In [4]:
tsne = TSNE(n_components=2,
            perplexity=30,
            learning_rate=200,
            max_iter=1000,
            random_state=42,
            init="pca")
coords = tsne.fit_transform(embeddings)

df["x"] = coords[:,0]
df["y"] = coords[:,1]

def cutoff(n): return lambda x: x if len(x) < n else x[:n] + "..."
cut = 100
df["nl"] = df["nl"].map(cutoff(cut))
df["bool"] = df["bool"].map(cutoff(cut))

fig = px.scatter(
    df, x="x", y="y",
    color="source",               # color by data source
    # size="quality",               # optionally size by quality score
    hover_data={
        "nl": True,
        "bool": True,
        "quality": True,
        "x": False, "y": False
    },
    title=f"t-SNE of {N} Text Embeddings"
)

fig.update_traces(marker=dict(opacity=0.7, line_width=0.5))
fig.update_layout(width=800, height=600)
fig.show()